In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal, Annotated
from langchain_core.messages import HumanMessage, SystemMessage
import operator

In [ ]:
generator_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
evaluator_llm = ChatOpenAI(model_name="gpt-4", temperature=0)
optimizer_llm = ChatOpenAI(model_name="gpt-4", temperature=0)


In [ ]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [ ]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [ ]:
class tweetState (TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["Approved", "Needs Improvement"]
    feedback: str
    iterations: int
    max_iterations: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [ ]:
def generator(state: tweetState) -> tweetState:

    #prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
        Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

        Rules:
        - Do NOT use question-answer format.
        - Max 280 characters.
        - Use observational humor, irony, sarcasm, or cultural references.
        - Think in meme logic, punchlines, or relatable takes.
        - Use simple, day to day english
        """)]

    #send to generator_llm
    response = generator_llm.invoke(messages).content

    #return response

    return {"tweet": response, "tweet_history": [response]}

In [ ]:
def evaluator(state: tweetState) -> tweetState:

    #prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback], "feedback_history": [response.feedback]}

In [ ]:
def optimizer(state: tweetState) -> tweetState:

    #prompt
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [ ]:
def route_evaluation(state: tweetState):
    if state['evaluation'] == "Approved" or state['iterations'] >= state['max_iterations']:
        return "Approved"
    else:
        return "Needs Improvement"

In [ ]:
graph = StateGraph(tweetState)

graph.add_node("generator", generator)
graph.add_node("evaluator", evaluator)
graph.add_node("optimizer", optimizer)

graph.add_edge(START, "generator")
graph.add_edge("generator", "evaluator")

graph.add_conditional_edges("evaluator", route_evaluation, { "Approved": END, "Needs Improvement": "optimizer" })
graph.add_edge("optimizer", "evaluator")

workflow = graph.compile()

In [ ]:
initial_state = {
    "topic": "The quirks of working from home",
    "iterations": 1,
    "max_iterations": 5,
}
workflow.invoke(initial_state)